In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import sklearn as sk
import sys
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RANSACRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm
import statsmodels.formula.api as smf

boston_data = load_boston()
df = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
print(df.head())
print(df.shape)

X = df
y = boston_data.target

print(y)

# 1. Utilizando statsmodels.api
intercept = sm.add_constant(X)
constantDf = pd.DataFrame(intercept)
print(constantDf)

model = sm.OLS(y, intercept)

lr = model.fit()
modelSummary = lr.summary()
print(modelSummary)

# 2. Utilizando statsmodel formula
form_lr = smf.ols(formula='y ~ CRIM + ZN + CHAS + NOX + RM + DIS + RAD + TAX + PTRATIO + B + LSTAT', data=df)
mlr = form_lr.fit()

modelFSummary = mlr.summary()

print(modelFSummary)

# --------------------------------- Ejercicio regresion Multiple con pruebas de especificación ---------------------------------

form_lr = smf.ols(formula='y ~ CRIM + ZN + CHAS + NOX', data=df)
mlr = form_lr.fit()
print(mlr.summary())


# Multicolinealidad
pd.options.display.float_format = '{:,.4f}'.format
corr_matrix = df.corr()

print(corr_matrix)

corr_matrix[np.abs(corr_matrix) < 0.6] = 0

plt.figure(figsize=(16, 10))
sns.heatmap(corr_matrix, annot=True, cmap='YlGnBu')
plt.show()

# b) Multicolinealidad con eigenvalores

eigenvalues, eigenvectors = np.linalg.eig(df.corr())

valoresPropios = pd.Series(eigenvalues).sort_values()
print(valoresPropios)

# El valor en posición 8 es muy pequeño comparado con los demás lo que significa que hay presencia de Multicolinealidad.

EigenVectors = np.abs(pd.Series(eigenvectors[:, 8])).sort_values(ascending=False)
print(EigenVectors)

'''
# Small eigenvalues or Large EigenVectors
# Es decir, si los eigenvalores son pequeños entonces hay presencia de Multicolinealidad, si los eigenvectores son grandes
# entonces hay presencia de Multicolinealidad
'''


# 2. Revisando la importancia de las propiedades o variables independientes del modelo (Cuál es la mejor para utilizar en el modelo)
# Este código lo que hace es mostrar un listado con los coeficientes obtenidos de cada variable y de este modo ver cuál explica más a la variable dependiente

df.head()

plt.hist(df['TAX'])
plt.hist(df['NOX'])

# 3. Estandarizar variable para identificar
model = LinearRegression()
model.fit(X, y)

result = pd.DataFrame(list(zip(model.coef_, df.columns)), columns=['coefficient', 'name']).set_index('name')
np.abs(result).sort_values(by='coefficient', ascending=False)

scaler = StandardScaler()
standard_coefficient_linear_reg = make_pipeline(scaler, model)

standard_coefficient_linear_reg.fit(X, y)

result = pd.DataFrame(list(zip(standard_coefficient_linear_reg.steps[1][1].coef_, df.columns)), columns=['coefficient', 'name']).set_index('name')

np.abs(result).sort_values(by='coefficient', ascending=False)


# 3. Utilizar la R2 para evaluar las variables claves del modelo.
# Se compara la R2 con y sin la variable de estudio. Si la R2 cambia drásticamente significa que esa variable es la correcta

linear_reg = smf.ols(formula='y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT', data=df)
benchmark = linear_reg.fit()
r2_score(y, benchmark.predict(df))


linear_reg = smf.ols(formula='y ~ CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B', data=df)
lr_noLSTAT = linear_reg.fit()
r2_score(y, lr_noLSTAT.predict(df))
